In [1]:
import cv2
from imutils import contours
import numpy as np
import pynput

In [2]:
image = cv2.imread('1.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imwrite("2-1.png", gray)
thresh = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,57,5)
cv2.imwrite("2.png", thresh)

True

In [3]:
# Filter out all numbers and noise to isolate only boxes
cnts = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 1000:
        cv2.drawContours(thresh, [c], -1, (0,0,0), -1)
        
cv2.imwrite("3.png", thresh)

True

In [4]:
# Fix horizontal and vertical lines
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,5))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, vertical_kernel, iterations=9)
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, horizontal_kernel, iterations=4)

cv2.imwrite("4.png", thresh)

# Sort by top to bottom and each row by left to right
invert = 255 - thresh
cnts = cv2.findContours(invert, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
(cnts, _) = contours.sort_contours(cnts, method="top-to-bottom")

rows = []
row = []
for (i, c) in enumerate(cnts, 1):
    area = cv2.contourArea(c)
    if area < 50000:
        row.append(c)
        if i % 8 == 0:  
            (cnts, _) = contours.sort_contours(row, method="left-to-right")
            rows.append(cnts)
            row = []


# Iterate through each box
# for row in rows:
#     for c in row:
#         mask = np.zeros(image.shape, dtype=np.uint8)
#         cv2.drawContours(mask, [c], -1, (255,255,255), -1)
#         result = cv2.bitwise_and(image, mask)
#         result[mask==0] = 255
#         cv2.imshow('result', result)
#         cv2.waitKey(175)

cv2.imwrite('thresh.png', thresh)
cv2.imwrite('invert.png', invert)


True

In [5]:
def draw_grid(img, grid_shape, color=(0, 255, 0), thickness=1):
    h, w, _ = img.shape
    rows, cols = grid_shape
    dy, dx = h / rows, w / cols

    # draw vertical lines
    for x in np.linspace(start=dx, stop=w-dx, num=cols-1):
        x = int(round(x))
        cv2.line(img, (x, 0), (x, h), color=color, thickness=thickness)

    # draw horizontal lines
    for y in np.linspace(start=dy, stop=h-dy, num=rows-1):
        y = int(round(y))
        cv2.line(img, (0, y), (w, y), color=color, thickness=thickness)

    return img

In [15]:
grid_image = cv2.imread('3.png')
draw_grid(grid_image, (8,8))
grid_image = cv2.imwrite('3.png', grid_image)
grid_image = cv2.imread('1-1.png')

def slice_board(board, nslices):
    h, w, channels = board.shape
    slicesY = [(h//nslices)*n for n in range(1,nslices+1)]
    slicesX = [(w//nslices)*n for n in range(1,nslices+1)]
    tiles = []
    prevY = 0
    for y in slicesY:
        row = board[prevY:y, :]
        prevY = y
        prevX = 0
        for x in slicesX:
            tile = row[:, prevX:x]
            tiles.append(tile)
            prevX = x
    return tiles
tiles = slice_board(grid_image, 8)
for each in range(0, len(tiles)):
    cv2.imshow('tile'+str(each)+'.png', tiles[each]) 
    cv2.waitKey(0)


In [ ]:
# from functools import partial
# from PIL import Image
# from pynput import mouse
# import numpy as np
# import io
# import cv2 


# memoryBuffer = io.BytesIO() 
# Image.open("1.png").save(memoryBuffer, format='png')
# bytes_as_np_array = np.frombuffer(memoryBuffer.getvalue(), dtype=np.uint8)
# temp = cv2.imdecode(bytes_as_np_array, cv2.IMREAD_ANYCOLOR)
# cv2.imshow("tempo", temp)
# cv2.waitKey(0)

# memoryBuffer = io.BytesIO() 
# Image.open("2.png").save(memoryBuffer, format='png')
# bytes_as_np_array = np.frombuffer(memoryBuffer.getvalue(), dtype=np.uint8)
# temp = cv2.imdecode(bytes_as_np_array, cv2.IMREAD_ANYCOLOR)
# cv2.imshow("tempo", temp)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
